In [ ]:
!nvidia-smi

Wed Mar 10 16:39:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Experiment 3

Now, we'll take a different approach which will result in increased number of samples. We'll use a *'sliding window'* technique that will slide over the samples and partition the sample into encoder and decoder data.

For example, if `sliding_window=4` then for a sample of length=25, first 4 timesteps will go to encoder_data and rest would be decoder_data. The window will then slide, lets say 1 timestep to the right, making timesteps 2-5 the encoder data and rest as decoder_data. This way, the sliding window will be shifted through the entire sample until it touches the threshold for minimum decoder_data_length.

By designing the samples in this way, we hope to better capture the relationship between the attack steps and thus increasing the accuracy of prediction.

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Thesis/Attack\ Step\ Prediction/Implementation

/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation


In [ ]:
import tensorflow as tf
import numpy as np
import random
import csv

import aptgen_utils

In [ ]:
# to reflect the changes, the module needs to be reloaded
import importlib
importlib.reload(aptgen_utils)

<module 'aptgen_utils' from '/content/drive/MyDrive/Thesis/Attack Step Prediction/Implementation/aptgen_utils.py'>

In [ ]:
# get attack sequences as list of lists
data_text = aptgen_utils.get_data_text()
print(data_text[0])
len(data_text)

['collection TA0009 Email_Collection T1114', 'credential_access TA0006 Credential_Dumping T1003', 'discovery TA0007 System_Information_Discovery T1082', 'collection TA0009 Email_Collection T1114', 'defense_evasion TA0005 File_Deletion T1107', 'persistence TA0003 Scheduled_Task T1053', 'discovery TA0007 System_Information_Discovery T1082', 'collection TA0009 Email_Collection T1114']


800

In [ ]:
# strip all spaces from data_text
data_text = [[''.join(''.join(step).split()) for step in data] for data in data_text]
print(data_text[0])
len(data_text)

['collectionTA0009Email_CollectionT1114', 'credential_accessTA0006Credential_DumpingT1003', 'discoveryTA0007System_Information_DiscoveryT1082', 'collectionTA0009Email_CollectionT1114', 'defense_evasionTA0005File_DeletionT1107', 'persistenceTA0003Scheduled_TaskT1053', 'discoveryTA0007System_Information_DiscoveryT1082', 'collectionTA0009Email_CollectionT1114']


800

Lets look at the sequence lengths in the data.

In [ ]:
from collections import Counter

# get length of different sequences in a list
sequence_lengths = [len(sequence) for sequence in data_text]

counter = Counter(sequence_lengths)
print("Frequncy of sequence lengths:", counter)
print("Number of unique sequence lengths:", len(counter.keys()))

Frequncy of sequence lengths: Counter({8: 117, 6: 62, 14: 56, 12: 49, 15: 45, 9: 44, 16: 42, 7: 39, 13: 37, 11: 32, 18: 30, 5: 29, 10: 29, 17: 28, 22: 23, 19: 19, 21: 18, 25: 14, 20: 10, 27: 9, 28: 9, 24: 8, 29: 8, 26: 7, 30: 7, 32: 7, 23: 6, 4: 5, 33: 4, 31: 3, 34: 3, 35: 1})
Number of unique sequence lengths: 32


In [ ]:
counter

Counter({4: 5,
         5: 29,
         6: 62,
         7: 39,
         8: 117,
         9: 44,
         10: 29,
         11: 32,
         12: 49,
         13: 37,
         14: 56,
         15: 45,
         16: 42,
         17: 28,
         18: 30,
         19: 19,
         20: 10,
         21: 18,
         22: 23,
         23: 6,
         24: 8,
         25: 14,
         26: 7,
         27: 9,
         28: 9,
         29: 8,
         30: 7,
         31: 3,
         32: 7,
         33: 4,
         34: 3,
         35: 1})

We'll work with a sliding window of 4 to start with, so all samples with length 4 will be removed.

In [ ]:
def remove_sequence(data_text, lengths_to_remove):
    lengths_to_remove = lengths_to_remove
    truncated_data_text = [sequence for sequence in data_text if len(sequence) not in lengths_to_remove]
    print("First sequence after truncating:", truncated_data_text[0])
    print("Number of sequence after truncating:", len(truncated_data_text))

    return truncated_data_text

In [ ]:
# remove sequences having length 4
lengths_to_remove = [4]
truncated_data_text = remove_sequence(data_text, lengths_to_remove)

First sequence after truncating: ['collectionTA0009Email_CollectionT1114', 'credential_accessTA0006Credential_DumpingT1003', 'discoveryTA0007System_Information_DiscoveryT1082', 'collectionTA0009Email_CollectionT1114', 'defense_evasionTA0005File_DeletionT1107', 'persistenceTA0003Scheduled_TaskT1053', 'discoveryTA0007System_Information_DiscoveryT1082', 'collectionTA0009Email_CollectionT1114']
Number of sequence after truncating: 795


In [ ]:
def get_encoder_decoder_text(text, sliding_window_size, shift_right, min_decoder_length):
    encoder_text, decoder_text = [], []

    for sequence in text:
        # the window will slide 'shift_right' positions at each iteration
        for i in range(0, len(sequence) - sliding_window_size, shift_right):
            # break if decoder_text length exceeds min_decoder_length
            if i+sliding_window_size+min_decoder_length > len(sequence):
                break

            encoder_text.append(sequence[i:i+sliding_window_size])
            decoder_text.append(sequence[i+sliding_window_size:])

    return encoder_text, decoder_text

In [ ]:
sliding_window_size = 5
shift_right = 1
min_decoder_length = 1

encoder_text, decoder_text = get_encoder_decoder_text(truncated_data_text, sliding_window_size, shift_right, min_decoder_length)
print("Number of samples:", len(encoder_text))

Number of samples: 6835


In [ ]:
def append_sos_eos(decoder_text):    
    decoder_input_text, decoder_target_text = [], []

    for sequence in decoder_text:
        decoder_input_text.append(["<sos>"] + sequence[:])
        decoder_target_text.append(sequence[:] + ["<eos>"])

    return decoder_input_text, decoder_target_text

In [ ]:
import copy
encoder_input_text = copy.deepcopy(encoder_text)
decoder_input_text, decoder_target_text = append_sos_eos(decoder_text)

print("encoder_input_text[0]:", encoder_input_text[0], "\ndecoder_input_text[0]:", decoder_input_text[0], "\ndecoder_target_text[0]:", decoder_target_text[0], "\n")
print("encoder_input_text[69]:", encoder_input_text[69], "\ndecoder_input_text[69]:", decoder_input_text[69], "\ndecoder_target_text[69]:", decoder_target_text[69], "\n")
print("encoder_input_text[169]:", encoder_input_text[169], "\ndecoder_input_text[169]:", decoder_input_text[169], "\ndecoder_target_text[169]:", decoder_target_text[169], "\n")
print("encoder_input_text[650]:", encoder_input_text[650], "\ndecoder_input_text[650]:", decoder_input_text[650], "\ndecoder_target_text[650]:", decoder_target_text[650], "\n")

encoder_input_text[0]: ['collectionTA0009Email_CollectionT1114', 'credential_accessTA0006Credential_DumpingT1003', 'discoveryTA0007System_Information_DiscoveryT1082', 'collectionTA0009Email_CollectionT1114', 'defense_evasionTA0005File_DeletionT1107'] 
decoder_input_text[0]: ['<sos>', 'persistenceTA0003Scheduled_TaskT1053', 'discoveryTA0007System_Information_DiscoveryT1082', 'collectionTA0009Email_CollectionT1114'] 
decoder_target_text[0]: ['persistenceTA0003Scheduled_TaskT1053', 'discoveryTA0007System_Information_DiscoveryT1082', 'collectionTA0009Email_CollectionT1114', '<eos>'] 

encoder_input_text[69]: ['discoveryTA0007Permission_Groups_DiscoveryT1069', 'defense_evasionTA0005File_DeletionT1107', 'persistenceTA0003Scheduled_TaskT1053', 'discoveryTA0007System_Information_DiscoveryT1082', 'discoveryTA0007Permission_Groups_DiscoveryT1069'] 
decoder_input_text[69]: ['<sos>', 'discoveryTA0007Remote_System_DiscoveryT1018', 'collectionTA0009Data_StagedT1074', 'lateral_movementTA0008Pass_the_

# Text sequences to integer sequences

In [ ]:
from keras.preprocessing.text import Tokenizer

def get_tokenizer(text):
    tokenizer = Tokenizer(num_words=200, lower=False) # we just give a large enough arbitrary number
    tokenizer.fit_on_texts(text)
    
    # builid word2idx and idx2word dictionary
    word2idx = copy.deepcopy(tokenizer.word_index)
    idx2word = {v:k for k, v in tokenizer.word_index.items()}

    return tokenizer, word2idx, idx2word

In [ ]:
def get_encoder_decoder_indices(encoder_input_text, decoder_input_text, decoder_target_text):
    encoder_tokenizer, encoder_word2idx, encoder_idx2word = get_tokenizer(encoder_input_text)
    print("encoder_word2idx:", encoder_word2idx)
    print("encoder_idx2word:", encoder_idx2word)
    encoder_input_indices = encoder_tokenizer.texts_to_sequences(encoder_input_text)


    decoder_tokenizer, decoder_word2idx, decoder_idx2word = get_tokenizer(decoder_input_text + decoder_target_text)
    print("\ndecoder_word2idx:", decoder_word2idx)
    print("decoder_idx2word:", decoder_idx2word)
    decoder_input_indices = decoder_tokenizer.texts_to_sequences(decoder_input_text)
    decoder_target_indices = decoder_tokenizer.texts_to_sequences(decoder_target_text)

    print("\nencoder_input_indices[0]:", encoder_input_indices[0], "\ndecoder_input_indices[0]:", decoder_input_indices[0], "\ndecoder_target_indices[0]:", decoder_target_indices[0], "\n")
    print("encoder_input_indices[69]:", encoder_input_indices[69], "\ndecoder_input_indices[69]:", decoder_input_indices[69], "\ndecoder_target_indices[69]:", decoder_target_indices[69], "\n")
    print("encoder_input_indices[169]:", encoder_input_indices[169], "\ndecoder_input_indices[169]:", decoder_input_indices[169], "\ndecoder_target_indices[169]:", decoder_target_indices[169], "\n")
    print("encoder_input_indices[650]:", encoder_input_indices[650], "\ndecoder_input_indices[650]:", decoder_input_indices[650], "\ndecoder_target_indices[650]:", decoder_target_indices[650], "\n")

    return encoder_input_indices, decoder_input_indices, decoder_target_indices, encoder_word2idx, encoder_idx2word, decoder_word2idx, decoder_idx2word

In [ ]:
# get input_indices and word conversion dicts
encoder_input_indices, decoder_input_indices, decoder_target_indices, encoder_word2idx, encoder_idx2word, decoder_word2idx, decoder_idx2word \
= \
get_encoder_decoder_indices(encoder_input_text, decoder_input_text, decoder_target_text)

encoder_word2idx: {'credential_accessTA0006Credential_DumpingT1003': 1, 'discoveryTA0007File_and_Directory_DiscoveryT1083': 2, 'collectionTA0009Data_StagedT1074': 3, 'defense_evasionTA0005File_DeletionT1107': 4, 'discoveryTA0007Remote_System_DiscoveryT1018': 5, 'defense_evasionTA0005Deobfuscate/Decode_Files_or_InformationT1140': 6, 'lateral_movementTA0008Remote_File_CopyT1105': 7, 'exfiltrationTA0010Exfiltration_Over_Command_and_Control_ChannelT1041': 8, 'collectionTA0009Email_CollectionT1114': 9, 'collectionTA0009Input_CaptureT1056': 10, 'discoveryTA0007System_Time_DiscoveryT1124': 11, 'credential_accessTA0006Credentials_in_FilesT1081': 12, 'defense_evasionTA0005Indicator_Removal_on_HostT1070': 13, 'credential_accessTA0006Input_CaptureT1056': 14, 'persistenceTA0003New_ServiceT1050': 15, 'credential_accessTA0006Credentials_in_RegistryT1214': 16, 'lateral_movementTA0008Remote_Desktop_ProtocolT1076': 17, 'persistenceTA0003Scheduled_TaskT1053': 18, 'discoveryTA0007Account_DiscoveryT1087':

# Padding, Split & One-hot 🔥

In [ ]:
max_encoder_seq_length = max([len(sequence) for sequence in encoder_input_indices])
max_decoder_seq_length = max([len(sequence) for sequence in decoder_input_indices])

print("Max sequence length for encoder:", max_encoder_seq_length)
print("Max sequence length for decoder:", max_decoder_seq_length)

Max sequence length for encoder: 5
Max sequence length for decoder: 31


In [ ]:
from keras.preprocessing.sequence import pad_sequences

def get_padded_inputs(encoder_input_indices, decoder_input_indices, decoder_target_indices):
    
    padded_decoder_input = pad_sequences(decoder_input_indices, maxlen=max_decoder_seq_length, dtype='int32', padding='post')
    padded_decoder_target = pad_sequences(decoder_target_indices, maxlen=max_decoder_seq_length, dtype='int32', padding='post')
    print("\npadded_decoder_input[0]", padded_decoder_input[0])
    print("padded_decoder_target[0]", padded_decoder_target[0])
    print("padded_decoder_input[69]", padded_decoder_input[69])
    print("padded_decoder_target[69", padded_decoder_target[69])

    return padded_decoder_input, padded_decoder_target

In [ ]:
padded_decoder_input, padded_decoder_target = get_padded_inputs(encoder_input_indices, decoder_input_indices, decoder_target_indices)


padded_decoder_input[0] [ 5 25 28 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0]
padded_decoder_target[0] [25 28 11  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0]
padded_decoder_input[69] [ 5  9  2 22  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0]
padded_decoder_target[69 [ 9  2 22  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0]


In [ ]:
def get_train_test_data(num_train_sample, encoder_input_indices, padded_decoder_input, padded_decoder_target):
    num_total_sample = len(encoder_input_indices)
    random.seed(69)
    random_indices = list(range(num_total_sample))
    random.shuffle(random_indices)
    # print(random_indices)
    encoder_train_input = [encoder_input_indices[i] for i in random_indices[:num_train_sample]]
    decoder_train_input = [padded_decoder_input[i] for i in random_indices[:num_train_sample]]
    decoder_train_target = [padded_decoder_target[i] for i in random_indices[:num_train_sample]]

    encoder_test_input = [encoder_input_indices[i] for i in random_indices[num_train_sample:]]
    decoder_test_input = [padded_decoder_input[i] for i in random_indices[num_train_sample:]]
    decoder_test_target = [padded_decoder_target[i] for i in random_indices[num_train_sample:]]

    print("Number of training samples:", len(encoder_train_input))
    print("Number of testing samples:", len(encoder_test_input))

    print("\nencoder_train_input[0]:", encoder_train_input[0], "\ndecoder_train_input[0]:", decoder_train_input[0], "\ndecoder_train_target[0]:", decoder_train_target[0], "\n")
    print("encoder_train_input[69]:", encoder_train_input[69], "\ndecoder_train_input[69]:", decoder_train_input[69], "\ndecoder_train_target[69]:", decoder_train_target[69], "\n")
    print("encoder_test_input[0]:", encoder_test_input[0], "\ndecoder_test_input[0]:", decoder_test_input[0], "\ndecoder_test_target[0]:", decoder_test_target[0], "\n")
    print("encoder_test_input[15]:", encoder_test_input[15], "\ndecoder_test_input[15]:", decoder_test_input[15], "\ndecoder_test_target[15]:", decoder_test_target[15], "\n")

    return encoder_train_input, decoder_train_input, decoder_train_target, encoder_test_input, decoder_test_input, decoder_test_target

In [ ]:
# split the data into train and test set
num_total_sample = len(encoder_input_indices)
num_train_sample = int(num_total_sample * 0.8)
encoder_train_input, decoder_train_input, decoder_train_target, encoder_test_input, decoder_test_input, decoder_test_target \
= \
get_train_test_data(num_train_sample, encoder_input_indices, padded_decoder_input, padded_decoder_target)

Number of training samples: 5468
Number of testing samples: 1367

encoder_train_input[0]: [26, 2, 3, 4, 34] 
decoder_train_input[0]: [ 5  2 30 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0] 
decoder_train_target[0]: [ 2 30 10  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0] 

encoder_train_input[69]: [6, 49, 5, 7, 1] 
decoder_train_input[69]: [ 5 21  7  2  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0] 
decoder_train_target[69]: [21  7  2  3  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0] 

encoder_test_input[0]: [30, 30, 3, 8, 18] 
decoder_test_input[0]: [ 5 26 28  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0] 
decoder_test_target[0]: [26 28  1  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0] 

encoder_test_input[15]: [1, 5, 4, 2, 3] 
decoder_test_input[15]: [ 5 36 1

In [ ]:
num_encoder_tokens = max(encoder_word2idx.values())
num_decoder_tokens = max(decoder_word2idx.values())

print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)

Number of unique input tokens: 50
Number of unique output tokens: 52


In [ ]:
def convert_to_onehot(encoder_train_input, decoder_train_input, decoder_train_target):    
    encoder_train_input_oh = tf.one_hot(encoder_train_input, num_encoder_tokens+1, dtype='int32').numpy() # +1 for 0s (were added for padding)
    decoder_train_input_oh = tf.one_hot(decoder_train_input, num_decoder_tokens+1, dtype='int32').numpy()
    decoder_train_target_oh = tf.one_hot(decoder_train_target, num_decoder_tokens+1, dtype='int32').numpy()

    return encoder_train_input_oh, decoder_train_input_oh, decoder_train_target_oh

In [ ]:
encoder_train_input_oh, decoder_train_input_oh, decoder_train_target_oh \
= \
convert_to_onehot(encoder_train_input, decoder_train_input, decoder_train_target)

print("encoder_train_input_oh shape:", encoder_train_input_oh.shape)
print("decoder_train_input_oh shape:", decoder_train_input_oh.shape)
print("decoder_train_target_oh shape:", decoder_train_target_oh.shape)

encoder_train_input_oh shape: (5468, 5, 51)
decoder_train_input_oh shape: (5468, 31, 53)
decoder_train_target_oh shape: (5468, 31, 53)


In [ ]:
encoder_train_input_oh[0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]], dtype=int32)

# Defining the training model

In [ ]:
from tensorflow.keras import Model, layers, Input
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.backend as K

from matplotlib import pyplot

In [ ]:
def plot_history(history):

    max_val_acc = max(history.history['val_accuracy'])
    max_val_acc_idx = np.argmax(history.history['val_accuracy'])
    print("Max Validation Accuracy", max_val_acc, " at epoch:", max_val_acc_idx+1, " with Training Accuracy", history.history['accuracy'][max_val_acc_idx])
    print()

    # plot metrics
    pyplot.plot(history.history['loss'])
    pyplot.plot(history.history['val_loss'])
    pyplot.show()
    pyplot.plot(history.history['accuracy'])
    pyplot.plot(history.history['val_accuracy'])
    pyplot.show()

In [ ]:
def get_stacked_model(n_units):

    K.clear_session()
    # credit: https://stackoverflow.com/a/56693548/7737870
    # latent_dims is an array which defines the depth of the encoder/decoder, as well as how large
    # the layers should be. So an array of sizes [a,b,c]  would produce a depth-3 encoder and decoder
    # with layer sizes equal to [c,b,a] and [a,b,c] respectively.
    encoder_inputs = Input(shape=(None, num_encoder_tokens+1), name="encoder_input")
    encoder_outputs = encoder_inputs
    encoder_states = []
    for j in range(len(n_units))[::-1]:
        encoder_outputs, h, c = layers.LSTM(n_units[j], return_state=True, return_sequences=bool(j), dropout=0.2, name=f"encoder_lstm_{len(n_units) - j}")(encoder_outputs)
        encoder_states += [h, c]

    decoder_inputs = Input(shape=(None, num_decoder_tokens+1), name="decoder_input")
    masked_decoder_inputs = layers.Masking(mask_value=0, name="decoder_masking")(decoder_inputs)
    decoder_outputs = masked_decoder_inputs
    output_layers = []
    for j in range(len(n_units)):
        output_layers.append( \
            layers.LSTM(n_units[len(n_units) - j - 1], return_sequences=True, return_state=True, name=f"decoder_lstm_{j+1}") \
        )
        decoder_outputs, dh, dc = output_layers[-1](decoder_outputs, initial_state=encoder_states[2*j:2*(j+1)])

    decoder_dense = layers.Dense(num_decoder_tokens+1, activation='softmax', name="decoder_dense")
    decoder_outputs = decoder_dense(decoder_outputs)

    # define training model
    stacked_model = Model([encoder_inputs, decoder_inputs], decoder_outputs, name="stacked_model")

    return stacked_model

In [ ]:
def get_stacked_model_with_dropout(n_units, dropout_at, dropout_value):

    K.clear_session()
    # credit: https://stackoverflow.com/a/56693548/7737870
    # latent_dims is an array which defines the depth of the encoder/decoder, as well as how large
    # the layers should be. So an array of sizes [a,b,c]  would produce a depth-3 encoder and decoder
    # with layer sizes equal to [c,b,a] and [a,b,c] respectively.
    encoder_inputs = Input(shape=(None, num_encoder_tokens+1), name="encoder_input")
    encoder_outputs = encoder_inputs
    encoder_states = []
    for j in range(len(n_units))[::-1]:
        encoder_outputs, h, c = layers.LSTM(n_units[j], return_state=True, return_sequences=bool(j), name=f"encoder_lstm_{len(n_units) - j}")(encoder_outputs)
        encoder_states += [h, c]

    decoder_inputs = Input(shape=(None, num_decoder_tokens+1), name="decoder_input")
    masked_decoder_inputs = layers.Masking(mask_value=0, name="decoder_masking")(decoder_inputs)
    decoder_outputs = masked_decoder_inputs
    output_layers = []


    dropout_idx = 0
    j = 0
    for i in range(len(n_units) + len(dropout_at)):
        if i == dropout_at[dropout_idx]:
            output_layers.append(layers.Dropout(dropout_value[dropout_idx]))
            decoder_outputs = output_layers[-1](decoder_outputs)
            if dropout_idx < len(dropout_at)-1:
                dropout_idx += 1
        else:
            output_layers.append( \
                layers.LSTM(n_units[len(n_units) - j - 1], return_sequences=True, return_state=True, name=f"decoder_lstm_{j+1}") \
            )
            decoder_outputs, dh, dc = output_layers[-1](decoder_outputs, initial_state=encoder_states[2*j:2*(j+1)])
            j += 1

    decoder_dense = layers.Dense(num_decoder_tokens+1, activation='softmax', name="decoder_dense")
    decoder_outputs = decoder_dense(decoder_outputs)

    # define training model
    stacked_model = Model([encoder_inputs, decoder_inputs], decoder_outputs, name="stacked_model")

    return stacked_model

In [ ]:
# this model will pass context vector from only the last encoder layer to the first decoder layer
def get_stacked_model_last_state_only(n_units):

    K.clear_session()
    # credit: https://stackoverflow.com/a/56693548/7737870
    # latent_dims is an array which defines the depth of the encoder/decoder, as well as how large
    # the layers should be. So an array of sizes [a,b,c]  would produce a depth-3 encoder and decoder
    # with layer sizes equal to [c,b,a] and [a,b,c] respectively.

    # define encoder
    encoder_inputs = Input(shape=(None, num_encoder_tokens+1), name="encoder_input")
    encoder_outputs = encoder_inputs
    for j in range(len(n_units))[::-1]:
        encoder_outputs, h, c = layers.LSTM(n_units[j], return_state=True, return_sequences=bool(j), name=f"encoder_lstm_{len(n_units) - j}")(encoder_outputs)
    encoder_states = [h, c]

    # define decoder
    decoder_inputs = Input(shape=(None, num_decoder_tokens+1), name="decoder_input")
    masked_decoder_inputs = layers.Masking(mask_value=0, name="decoder_masking")(decoder_inputs)
    decoder_outputs = masked_decoder_inputs
    output_layers = []

    # add context vector as the first decoder initial state
    output_layers.append( \
            layers.LSTM(n_units[len(n_units) - 1], return_sequences=True, return_state=True, name=f"decoder_lstm_1") \
        )
    decoder_outputs, dh, dc = output_layers[-1](decoder_outputs, initial_state=encoder_states)
    # add other decoder layers
    for j in range(1, len(n_units)):
        output_layers.append( \
            layers.LSTM(n_units[len(n_units) - j - 1], return_sequences=True, return_state=True, name=f"decoder_lstm_{j+1}") \
        )
        decoder_outputs, dh, dc = output_layers[-1](decoder_outputs)

    decoder_dense = layers.Dense(num_decoder_tokens+1, activation='softmax', name="decoder_dense")
    decoder_outputs = decoder_dense(decoder_outputs)

    # define training model
    stacked_model = Model([encoder_inputs, decoder_inputs], decoder_outputs, name="stacked_model")

    return stacked_model

In [ ]:
n_units = [1024, 800, 128]
# dropout_at = [1]
# dropout_value = [0.2]
stacked_model = get_stacked_model(n_units)
# stacked_model = get_stacked_model_with_dropout(n_units, dropout_at, dropout_value)
stacked_model.summary()

# compile
stacked_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "stacked_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      [(None, None, 53)]   0                                            
__________________________________________________________________________________________________
encoder_input (InputLayer)      [(None, None, 51)]   0                                            
__________________________________________________________________________________________________
decoder_masking (Masking)       (None, None, 53)     0           decoder_input[0][0]              
__________________________________________________________________________________________________
encoder_lstm_1 (LSTM)           [(None, None, 128),  92160       encoder_input[0][0]              
______________________________________________________________________________________

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(stacked_model)

In [ ]:
# # Subclass ModelCheckpoint
# class MyModelCheckpoint(ModelCheckpoint):

#     def __init__(self, *args, **kwargs):
#         super(MyModelCheckpoint, self).__init__(*args, **kwargs)


#     # redefine the save so it only activates after 200 epochs
#     def on_epoch_end(self, epoch, logs=None):
#         if epoch > 150: super(MyModelCheckpoint, self).on_epoch_end(epoch, logs)

# chkpoint_filepath = "(5_enc,3_layer,1024_800_128_units)augmented_s2s_stacked"
# model_checkpoint_callback = MyModelCheckpoint(
#     filepath=chkpoint_filepath,
#     save_best_only=True,
#     monitor='val_accuracy',
#     mode='max',
#     verbose=0
# )

In [ ]:
# K.clear_session()
# # fit
# history = stacked_model.fit( \
#             [encoder_train_input_oh, decoder_train_input_oh], decoder_train_target_oh, \
#             batch_size=64, \
#             epochs=500, \
#             validation_split=0.1,
#             callbacks=model_checkpoint_callback
#         )
# # stacked_model.save("(5_enc,1_layer,32_units)augmented_s2s_stacked")

In [ ]:
# Subclass ModelCheckpoint
class MyModelCheckpoint(ModelCheckpoint):

    def __init__(self, *args, **kwargs):
        super(MyModelCheckpoint, self).__init__(*args, **kwargs)


    # redefine the save so it only activates after a number of epochs
    def on_epoch_end(self, epoch, logs=None):
        if epoch > 150: super(MyModelCheckpoint, self).on_epoch_end(epoch, logs)

chkpoint_filepath = "(5_enc,3_layer,1024_800_128_units)augmented_s2s_stacked"
model_checkpoint_callback = MyModelCheckpoint(
    filepath=chkpoint_filepath,
    save_best_only=True,
    monitor='accuracy',
    mode='max',
    verbose=0
)

In [ ]:
K.clear_session()
# fit
history = stacked_model.fit( \
            [encoder_train_input_oh, decoder_train_input_oh], decoder_train_target_oh, \
            batch_size=64, \
            epochs=500, \
            callbacks=model_checkpoint_callback
        )
# stacked_model.save("(5_enc,1_layer,32_units)augmented_s2s_stacked")

Epoch 1/500
86/86 [==============================] - 48s 62ms/step - loss: 1.4442 - accuracy: 0.6934
Epoch 2/500
86/86 [==============================] - 5s 60ms/step - loss: 0.8246 - accuracy: 0.7723
Epoch 3/500
86/86 [==============================] - 5s 61ms/step - loss: 0.7449 - accuracy: 0.7933
Epoch 4/500
86/86 [==============================] - 5s 61ms/step - loss: 0.7063 - accuracy: 0.7981
Epoch 5/500
86/86 [==============================] - 5s 61ms/step - loss: 0.6680 - accuracy: 0.8035
Epoch 6/500
86/86 [==============================] - 5s 61ms/step - loss: 0.6134 - accuracy: 0.8167
Epoch 7/500
86/86 [==============================] - 5s 61ms/step - loss: 0.5726 - accuracy: 0.8273
Epoch 8/500
86/86 [==============================] - 5s 61ms/step - loss: 0.5169 - accuracy: 0.8452
Epoch 9/500
86/86 [==============================] - 5s 62ms/step - loss: 0.4605 - accuracy: 0.8634
Epoch 10/500
86/86 [==============================] - 5s 61ms/step - loss: 0.4021 - accuracy: 0.881

INFO:tensorflow:Assets written to: (5_enc,3_layer,1024_800_128_units)augmented_s2s_stacked/assets


INFO:tensorflow:Assets written to: (5_enc,3_layer,1024_800_128_units)augmented_s2s_stacked/assets


Epoch 153/500
86/86 [==============================] - 5s 61ms/step - loss: 0.0243 - accuracy: 0.9928


INFO:tensorflow:Assets written to: (5_enc,3_layer,1024_800_128_units)augmented_s2s_stacked/assets


...

INFO:tensorflow:Assets written to: (5_enc,3_layer,1024_800_128_units)augmented_s2s_stacked/assets


Epoch 466/500
86/86 [==============================] - 5s 63ms/step - loss: 0.0182 - accuracy: 0.9941
Epoch 467/500
86/86 [==============================] - 5s 63ms/step - loss: 0.0180 - accuracy: 0.9946
Epoch 468/500
86/86 [==============================] - 5s 64ms/step - loss: 0.0182 - accuracy: 0.9943
Epoch 469/500
86/86 [==============================] - 5s 63ms/step - loss: 0.0168 - accuracy: 0.9946


INFO:tensorflow:Assets written to: (5_enc,3_layer,1024_800_128_units)augmented_s2s_stacked/assets


INFO:tensorflow:Assets written to: (5_enc,3_layer,1024_800_128_units)augmented_s2s_stacked/assets


Epoch 470/500
86/86 [==============================] - 5s 63ms/step - loss: 0.0192 - accuracy: 0.9942
Epoch 471/500
86/86 [==============================] - 5s 64ms/step - loss: 0.0183 - accuracy: 0.9941
Epoch 472/500
86/86 [==============================] - 6s 65ms/step - loss: 0.0168 - accuracy: 0.9945
Epoch 473/500
86/86 [==============================] - 5s 63ms/step - loss: 0.0181 - accuracy: 0.9942
Epoch 474/500
86/86 [==============================] - 5s 63ms/step - loss: 0.0180 - accuracy: 0.9945
Epoch 475/500
86/86 [==============================] - 5s 63ms/step - loss: 0.0176 - accuracy: 0.9946
Epoch 476/500
86/86 [==============================] - 5s 63ms/step - loss: 0.0190 - accuracy: 0.9940
Epoch 477/500
86/86 [==============================] - 5s 63ms/step - loss: 0.0196 - accuracy: 0.9938
Epoch 478/500
86/86 [==============================] - 5s 62ms/step - loss: 0.0166 - accuracy: 0.9947
Epoch 479/500
86/86 [==============================] - 5s 63ms/step - loss: 0.0184

In [ ]:
max_acc = max(history.history['accuracy'])
max_acc_idx = np.argmax(history.history['accuracy'])
print("Max Accuracy", max_acc, " at epoch:", max_acc_idx+1)

pyplot.plot(history.history['loss'])
pyplot.show()
pyplot.plot(history.history['accuracy'])
pyplot.show()

NameError: ignored

In [ ]:
# plot_history(history)

KeyError: ignored

# Inference mode

In [ ]:
def get_inference_model(model):
    # define inference encoder
    encoder_inputs = model.input[0]
    encoder_states = []
    for i in range(1, len(n_units)+1):
        encoder_states += model.get_layer(f'encoder_lstm_{i}').output[1:]
    infer_encoder_model = Model(encoder_inputs, encoder_states)


    # define inference decoder
    decoder_inputs = model.input[1]
    decoder_outputs = decoder_inputs
    decoder_states_inputs = []
    decoder_states = []
    for j in range(len(n_units))[::-1]:
        current_state_inputs = [Input(shape=(n_units[j],)) for _ in range(2)]
        temp = model.get_layer(f'decoder_lstm_{len(n_units)-j}')(decoder_outputs, initial_state=current_state_inputs)
        decoder_outputs, curr_states = temp[0], temp[1:]

        decoder_states += curr_states
        decoder_states_inputs += current_state_inputs

    decoder_dense = model.get_layer('decoder_dense')
    decoder_outputs = decoder_dense(decoder_outputs)
    infer_decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states, name="augmented_s2s_model")
    
    return infer_encoder_model, infer_decoder_model

In [ ]:
def decode_sequence(infer_encoder_model, infer_decoder_model, input_seq, actual_target_sequence):
    # print(np.count_nonzero(actual_target_sequence))
    # if np.count_nonzero(actual_target_sequence) != 5:
    #     return -1

    input_seq_oh = tf.one_hot(input_seq, num_encoder_tokens+1, dtype='int32').numpy()

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens+1))
    # Populate the first character of target sequence with the start character <sos>
    target_seq[0, 0, decoder_word2idx['<sos>']] = 1

    # Encode the input as state vectors.
    states_value = infer_encoder_model.predict(input_seq_oh)

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sequence = []
    # print("Actual Input Sequence:", input_seq)
    # print("Text:", convert_to_seq(input_seq[0], encoder_idx2word), "\n")
    # print("Actual Target Sequence:", actual_target_sequence)
    # print("Text:", convert_to_seq(actual_target_sequence, decoder_idx2word), "\n")
    idx = 0

    while not stop_condition:
        #print(target_seq)
        # output_tokens, h, c = infer_decoder_model.predict([target_seq] + states_value)
        to_split = infer_decoder_model.predict([target_seq] + states_value)
        output_tokens, states_value = to_split[0], to_split[1:]

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, 0])
        decoded_sequence.append(sampled_token_index)
        #sampled_step = decoder_idx2word[sampled_token_index]
        
        # Exit condition: either hit max length
        # or find stop character <eos>
        if actual_target_sequence[idx] == decoder_word2idx['<eos>']:
            stop_condition = True

        # Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1, num_decoder_tokens+1))
        # print("actual:", actual_target_sequence[idx])
        # print("predicted:", sampled_token_index)
        if sampled_token_index == actual_target_sequence[idx]:
            # print("match")
            target_seq[0, 0, sampled_token_index] = 1
        else:
            target_seq[0, 0, actual_target_sequence[idx]] = 1 # feed in the actual step in case of wrong prediction
        
        # Update states
        # states_value = [h, c]
        idx += 1

    # print("Predicted Sequence:", decoded_sequence)
    # print("Text:", convert_to_seq(decoded_sequence, decoder_idx2word), "\n")
    return decoded_sequence

In [ ]:
from tqdm import tqdm

In [ ]:
def predict(infer_encoder_model, infer_decoder_model, encoder_test_input, decoder_test_target): 
    predicted_sequences = []
    correct_steps = []
    total_step_count = 0
    accurate_prediction_count = 0

    num_test_samples = len(encoder_test_input)
    for i in tqdm(range(num_test_samples)):
    # for i in range(num_test_samples):
        input_seq = encoder_test_input[i:i+1]
        decoded_sequence = decode_sequence(infer_encoder_model, infer_decoder_model, input_seq, decoder_test_target[i])

        correct_steps = [i for i, j in zip(decoder_test_target[i], decoded_sequence) if i == j]
        curr_acc_pred_cnt = len(correct_steps)
        accurate_prediction_count += curr_acc_pred_cnt
        total_step_count += len(decoded_sequence)
        # print(curr_acc_pred_cnt, " out of ",  len(decoded_sequence)," step(s) correctly predicted")

        predicted_sequences.append(decoded_sequence)
        # print()

    print("\nTotal Predicted Steps:", total_step_count)
    print("Total Accurate Prediction:", accurate_prediction_count)

    return predicted_sequences

In [ ]:
def convert_to_seq(seq, idx2word):
# seq = [30, 30, 3, 8, 18]
    attack_sequence = []
    for step_idx in seq:
        if step_idx == 0:
            break
        attack_sequence.append(idx2word[step_idx])
    return attack_sequence

In [ ]:
from tensorflow.keras import models
model = models.load_model("(5_enc,3_layer,1024_800_128_units)augmented_s2s_stacked")

In [ ]:
infer_encoder_model, infer_decoder_model = get_inference_model(model)
infer_decoder_model.summary()
predicted_sequences = predict(infer_encoder_model, infer_decoder_model, encoder_test_input, decoder_test_target)

  0%|          | 0/1367 [00:00<?, ?it/s]

Model: "augmented_s2s_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      [(None, None, 53)]   0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
decoder_lstm_1 (LSTM)           [(None, None, 128),  93184       decoder_input[0][0]              
                                                                 input_1[0][0]  

100%|██████████| 1367/1367 [10:14<00:00,  2.22it/s]


Total Predicted Steps: 11583
Total Accurate Prediction: 10088
